## <h1><center> Data Preparation and Machine Learning </center></h1>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import cross_val_score     
from sklearn.metrics import confusion_matrix             
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression                     
from sklearn.tree import DecisionTreeClassifier

from IPython.display import Image

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Online retailer, eBay is providing an option of bidding to their customers globally. Bidding is employed to find the real price of items in the market based on the demand. The price offered by anyone participating in this process is termed as a 'bid'. Normal bids are classified as ‘0’ bids in the data set and anomalous bids as ‘1’. The goal is to use classification or clustering algorithms to predict the bids in the future.

**Data Dictionary**
* **Record ID**: Unique identifier of a record in the dataset.
* **Auction ID**: Unique identifier of an auction.
* **Bidder ID**: Unique identifier of a bidder.
* **Bidder Tendency**: A shill bidder participates exclusively in auctions of few sellers rather than a diversified lot. This is a collusive act involving the fraudulent seller and an accomplice.
* **Bidding Ratio**: A shill bidder participates more frequently to raise the auction price and attract higher bids from legitimate participants.
* **Successive Outbidding**: A shill bidder successively outbids himself even though he is the current winner to increase the price gradually with small consecutive increments.
* **Last Bidding**: A shill bidder becomes inactive at the last stage of the auction (more than 90\% of the auction duration) to avoid winning the auction.
* **Auction Bids**: Auctions with SB activities tend to have a much higher number of bids than the average of bids in concurrent auctions.
* **Auction Starting Price**: a shill bidder usually offers a small starting price to attract legitimate bidders into the auction.
* **Early Bidding**: A shill bidder tends to bid pretty early in the auction (less than 25\% of the auction duration) to get the attention of auction users.
* **Winning Ratio**: A shill bidder competes in many auctions but hardly wins any auctions.
* **Auction Duration**: How long an auction lasted.
* **Class**: 0 for normal behaviour bidding; 1 for otherwise.

## Content:
### 1. Bussiness Understanding;
### 2. Data Understanding;
### 3. Data preparation;
### 4. Modeling;
### 5. Evaluation;
### 6. Deployment;
### References.

For purpose of this project we are going to use established model for data mining - **CRISP-DM** (the cross-industry standard process for data mining). CRISP-DM project consists of 6 stages. Following one another, it helps in keeping sequence of project's steps. The defined stages of CRISP-DM are:
* Business Understanding;
* Data understanding;
* Data Preparation;
* Modeling;
* Evaluation;
* Deployment.

### 1. Business understanding

The purpose of the Business Understanding phase is to understand what the business wants to solve. On this stage we set questions we need to answer with the project's outcome. Following the assigment task we can define the next questions within the project:
* What features are the most important for bidding class prediction?
* Can a target variable be predicted with an accuracy higher than 90%?

### 2. Data Understanding

The data understanding phase goes hand in hand with the business understanding phase and encourages the focus to ascertain, assemble, and scrutinize the data sets that can help us to achieve the project goals. (Trisit Kumar Chatterjee, 2020)

My first step will be uploading the dataset. Also, I want to set **Record_ID** as an index column. The reason for that is that the **Record_ID** column contains a unique identification number which can facilitate data retrieval and handling data in general.

In [ ]:
df = pd.read_csv('../input/shill-bidding-dataset/Shill Bidding Dataset.csv', index_col='Record_ID')

In [ ]:
df

Next, I want to see if we have any missing values and what type of the observations we have. I will use ***.info*** function.

In [ ]:
df.info()

We can see that overall the dataset consists of numerical data with one column represented by 'object' data type. Data size is 6321 rows and 12 columns. We also don't observe any missing values, which means we won't have to handle them in the next chapter. 

Next, I want to extract some statistical information and then get a visual understanding of the data and plot some graphs.

In [ ]:
df.describe()

From the table above we can observe, that values range from 0 to 1 for each feature except **Auction_Duration** and **Auction_ID**. **Last_Bidding** and **Auction_Duration** seem to follow a normal distribution. Also, I can infer that **Winning_Ratio** and **Successive_Outbidding** have lots of observations cluster around either **0** or **1** values. And the dependent variable **Class** is more represented by **0** class than **1** as mean equals 0.1. It means that the data is imbalanced and we will have to decide how to deal with it in the next chapter. 
Let's see how many observations we have represented for each class.

In [ ]:
sns.countplot(df['Class'])
print(plt.show())
print(df.Class.value_counts())

With the graph as well as with the ***value_counts*** we can confirm that **0** class has 5646 entries while **1** class has only 675 entries. I will see models' accuracy in the Modeling chapter and if imbalanced classes strongly affect accuracy we will have to balance them.

As the only one object feature we have is a bidder name I want to see if I have any unique values and then decide if we need to keep the feature.

In [ ]:
df.Bidder_ID.value_counts()

We can observe that we have 1054 unique values. We can't really extract any usefull information from it as it's represented by encrypted unique names, therefore I'll simply drop it.

In [ ]:
df = df.drop(columns='Bidder_ID')

As we are dealing with a classification problem we don't need to identify correlation between variables.

I also want to take a look at boxplots for each variable to get a better understanding of distribution and outliers.

In [ ]:
df.drop('Class', axis=1).plot(kind='box', subplots=True, layout=(6,2), sharex=False, sharey=False, figsize=(10,10), title='Box Plot for each input variable')
plt.show()

**Bidder_Tendency** and **Bidding_Ratio** have many outliers. We will discuss in the next chapter if we need to remove them. **Auction_ID**, **Last_Bidding**, **Early_Bidding** and **Auction_Duration** seem to follow normal distribution. We infered earlier that **Winning_Ratio** and **Starting_Price_Average** have values mostly clustered around **0** and **1**. **Successive_Outbidding** boxplot looks interesting, as it shows that we migh have outliers. I am going to plot density graphs for these 3 features to get a better visual comprehension.

In [ ]:
print(sns.kdeplot(data=df, x='Winning_Ratio', color='r', shade=True))
print(sns.kdeplot(data=df, x='Starting_Price_Average', color='y', shade=True))
print(sns.kdeplot(data=df, x='Successive_Outbidding', color='b', shade=True))

plt.xlabel('KDE')
plt.ylabel('Probability Density')

With the graph we can confirm the statement we made above. As for **Successive_Outbidding** variable, it's not ouliers but data is represented by only 3 values **0**, **0.5** and **1**. As number of **0** values are drastically higher than **0.5** and **1** it shows the values as outliers.

In [ ]:
print(df.Successive_Outbidding.value_counts())
print(df.Successive_Outbidding.describe())
print(sns.boxplot(data=df, x='Successive_Outbidding'))

We've prepared everything we needed and now can move to another stage **Data preparation**.

### 3. Data Preparation

This stage, which is often referred to as “data wrangling” or “data munging”, has the objective is to develop the final data set for modelling. Covers all activities to construct the final dataset from the initial raw data. Data preparation tasks are likely to be performed multiple times and not in any prescribed order. Tasks include table, record and attribute selection as well as transformation and cleaning of data for modelling tools.

I'll start the Data Preparation section with normalization. Normalization typically means rescaling the values into a range of [0,1]. (Geller, 2019)
When we have all features normalized within one range it will help ML models run faster and with the better accuracy.
The reason why I'll be using normalization is that most of the features are already ranged between 0 and 1 except **Auction_ID** and **Auction_Duration**, therefore I will scale only these two features.

In [ ]:
df1 = df[['Auction_ID','Auction_Duration']]
df2 = df.drop(columns = ['Auction_ID','Auction_Duration'])
minmax = MinMaxScaler().fit(df1)
minmax = minmax.transform(df1)

After I've applied the scaler I want to record the scaled array in DataFrame and then combine in one DataFrame.

In [ ]:
df2 = df2.reset_index()

In [ ]:
df1 = pd.DataFrame(minmax, columns = ['Auction_ID','Auction_Duration'])
df1

In [ ]:
df = df1.join(df2, how='outer')
index = df2['Record_ID']
df = df.set_index('Record_ID')
df

Finally, we've got the DataFrame with all feature ranged with the same range [0,1]. 

#### Features importances

As a part of my business understanding part I've defined a question I would like to get an answer *'What features are the most important for bidding class prediction?'*
In order to get an answer on the question I'll be using the ***DecisionTreeClassifier*** which will range fetures by their importance.
Feature importance is calculated as the decrease in node impurity weighted by the probability of reaching that node. The node probability can be calculated by the number of samples that reach the node, divided by the total number of samples. The higher the value the more important the feature. (Ronaghan, 2018)

As a first step of defining features importances I'll separate independent variables from the dependent variable.

In [ ]:
X = df.drop(columns = 'Class')
y = df['Class']

Next, I'll run the DecisionTreeClassifier.

In [ ]:
cls = DecisionTreeClassifier()
cls.fit(X, y)

In [ ]:
importances = cls.feature_importances_
columns = X.columns
columns

I want to plot a graph to display features importances in descending order.

In [ ]:
df_import = pd.DataFrame(importances, columns = ['Feature importances'], index=columns)
df_import['Percentage'] = pd.DataFrame((df_import.sort_values(by= ['Feature importances'], ascending=False)/df_import.shape[0])*100)
pd.options.display.float_format = "{:.2f}".format 
df_import=df_import.sort_values(by= ['Percentage'], ascending=False)

# plotting features importances
g = df_import['Percentage'].plot(kind = 'bar', figsize=(60,30), fontsize=24, color = 'pink')
g.set_title('Features importances', fontsize=70)
g.set_xlabel('Features', fontsize=50)
g.set_ylabel('Percentage of importances', fontsize=50)

We see that three the most important features for predicting the target variable are:
* ***Successive_Outbidding***;
* ***Auction_Duration***;
* ***Winning_Ratio***,

and the less important are:
* ***Starting_Price_Average***;
* ***Auction_ID***;
* ***Early_Bidding***.

#### Dimensionality reduction

We use dimensionality reduction when  we want to decrease the number of features in a dataset. Two techniques for dimensionality reduction are:
* Principal Component Analysis (PCA);
* Linear Discriminant Analysis (LDA).

Both LDA and PCA are linear transformation techniques: LDA is a supervised whereas PCA is unsupervised – PCA ignores class labels. In contrast to PCA, LDA attempts to find a feature subspace that maximizes class separability. (Raschka, 2022)
In this chapter I want to apply both PCA and LDA and compare results.

##### PCA

PCA is a statistical variance based procedure that converts a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components. In simpler words, PCA is often used to simplify data, reduce noise, and find unmeasured “latent variables”. (Volpi, 2020)
We've already separated independants variables and the target variable, therefore we can apply PCA straight away.

First, I want to plot an Explained varaince ratio graph to see how much of the variance in the original data is encapsulated in the new component variables.

In [ ]:
pca = PCA().fit(X)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.title('Explained variance ratio', fontsize = 20)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

From the graph above we see that in order to keep 90% of the original data we need to use 6 components.

In [ ]:
pca = PCA(n_components=6)
projected = pca.fit_transform(X)

In [ ]:
explained_variance = pca.explained_variance_ratio_
explained_variance

First 3 components contain the most ammount of data while the last three contain less than 20%. This the last step I want to create a DataFrame with 6 components and a target variable.

In [ ]:
principal_df = pd.DataFrame(data = projected, columns = ['PC 1','PC 2','PC 3','PC 4','PC 5','PC 6'], index=index)
df_pca = principal_df.join(y, how='outer')
df_pca.head(5)

##### LDA

Linear Discriminant Analysis, or LDA, uses the information from both features to create a new axis and projects the data on to the new axis in such a way as to minimizes the variance and maximizes the distance between the means of the two classes. (Maklin, 2019)

I start with ***fit_transform*** of the both dependent and independent variables and then will take a look at the explained_variance_ratio.

In [ ]:
lda = LinearDiscriminantAnalysis()
x_lda = lda.fit_transform(X,y)

In [ ]:
lda.explained_variance_ratio_

As our target variable is represented only by 2 classes, the LDA has reduced the number of components to 1. If we had at least two components we would be able to plot the data on the two-dimension graph. However in our case the data is represented by one dimension and the plot would look like a simple one dimension line.
I want to create a DataFrame and record there LDA 1 we got as well as the target variable.

In [ ]:
linear_df = pd.DataFrame(data = x_lda, columns = ['LDA 1'], index=index)
df_lda = linear_df.join(y, how='outer')
df_lda.head(10)

From the table above we see, that negative numbers in LDA 1 belong to '0 class' and positive numbers belong to '1 class'.

PCA dimensionality reduction method allows us to choose the number of components we need to use. Apart from PCA in LDA we can't control the number of components they will be always reduced by one from number of classes we have represented as a target variable. However, LDA is a better use for solving classification problems as it creates a clear separation between classes. 

It was the last step of this stage. The dataset is ready for applying ML models. For the purpose of the modeling stage I'm going to use dataset with applied LDA.

### 4. Modeling

At this stage we will select the actual modelling technique that is to be used. It can be one or several models. According to the CA task I'll choose two models to work with and will explain my choice.
As a first step of the stage I'll separate the target variable from dependent variables.

In [ ]:
X = df_lda.drop(columns='Class')
y = df_lda['Class']

Next, I'll split the dataset on train and test parts, keeping train part equals to 65% and train part 35%.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1, shuffle=True)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

The first algorithm I'll be working with is a Logistic Regression. Logistic Regression is a Supervised Machine Learning classification algorithm that is used to predict the probability of a categorical dependent variable. (Li, 2017).
The reason why I'll be using the Logistic regression algorithm is because it's designed for two-class (binary) problems, modeling the target using a binomial probability distribution function.

#### Logistic Regression

First step is to fit the model:

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

With the next step I want to define the best model's parameters in order to get the best possible accuracy.
I'll be working with the next LogisticRegression parameters:
* **solvers**;
* **penalty**;
* **c_values**.

**Solvers** - Algorithm to use in the optimization problem:
* For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones;
* For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss;
* ‘liblinear’ is limited to one-versus-rest schemes.

**Penalty** - Specify the norm of the penalty:
* 'none': no penalty is added;
* 'l2': add a L2 penalty term and it is the default choice;
* 'l1': add a L1 penalty term;

**C_values** - Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.

##### Tuning hyper-parameters of an estimator

We need to records parameters we've defined in a list:

In [ ]:
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
penalty = ['none','l1','l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

To tune the parameters we will be using GridSearchCV. GridSearchCV is as exhaustive search over specified parameter values for an estimator. Grid Search uses a different combination of all the specified hyperparameters and their values and calculates the performance for each combination and selects the best value for the hyperparameters. (Shah, 2021)

In [ ]:
grid = dict(solver=solvers,penalty=penalty,C=c_values)
grid_search = GridSearchCV(estimator=logreg, param_grid=grid, n_jobs=-1, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

We've defined that the best accuracy will'be reached by using C=0.1, penalty = 'l2' and a  solver = 'liblinear'. Now, let's train the model with these parameters:

In [ ]:
logreg = LogisticRegression(C=0.1, penalty='l2', solver='liblinear')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

Let's see what accuracy we get on training and testins set to make sure the model is not underfitted or overfitted.

In [ ]:
print('Training score: {:.3f}'.format(logreg.score(X_train,y_train)))
print('Testing score: {:.3f}'.format(logreg.score(X_test,y_test)))

Wee see that both training and testing scores are high and the model perfoms well on both sets. The last step is to run the classification report and identify ***Type I and Type II errors***.

In [ ]:
print(classification_report(y_test, y_pred))
print(sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='.4g',                 
                 xticklabels=['Normal behaviour (0 class)','Otherwise (1 class)'],                
                 yticklabels=['Normal behaviour (0 class)','Otherwise (1 class)'],                  
                 cbar=False, cmap='Blues'))

We can obserse that the model perfomed very well. Over 1984 **0 class** values 55 were predicted as False Positive and over 224 **1 class** values only 1 was predicted as False Negative, which is a very good result.

Next ML model I want to use is a DecisionTreeClassifier. Decision Tree is a Supervised Machine Learning Algorithm that uses a set of rules to make decisions. The intuition behind Decision Trees is that we use the dataset features to create yes/no questions and continually split the dataset until we isolate all data points belonging to each class. Every time we ask a question we add a node to the tree. The result of asking a question splits the dataset based on the value of a feature, and creates new nodes. The algorithm tries to completely separate the dataset such that all leaf nodes, i.e., the nodes that don’t split the data further, belong to a single class. These are called pure leaf nodes. In the end, the algorithm can only assign one class to the data points in each leaf node. (Bento, 2021)

#### Decision Tree

Fitting the model first:

In [ ]:
tree = DecisionTreeClassifier(random_state=1)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

Similarly to the previous chapter we want to define the best parameters in order to reach the best accuracy. For this model I'll be chechking the next parameters:
* **Max_depth**;
* **Min_samples_leaf**.

**Max_depth** - The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

**Min_samples_leaf** - The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. 

##### Tuning hyper-parameters of an estimator

For both hyperparameters defined I'll create a range between 1 to 20. The estimator will check each value of both in order to define the best ones.

In [ ]:
depth = range(1, 20)
leaf = range(1, 20)

In [ ]:
grid = dict(max_depth=depth,min_samples_leaf=leaf)
grid_search = GridSearchCV(estimator=tree, param_grid=grid, n_jobs=-1, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

We see that the estimator has defined **max_depth** and **min_samples_leaf** both equal 4. Let's train the model by using defined parameters.

In [ ]:
tree = DecisionTreeClassifier(max_depth=4, min_samples_leaf=4, random_state=1)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

In [ ]:
print('Training score: {:.3f}'.format(tree.score(X_train,y_train)))
print('Testing score: {:.3f}'.format(tree.score(X_test,y_test)))

Training and testing accuracy I've got are a bit higher than with the LogisticRegression and should give us a better result in the classification report.

In [ ]:
print(classification_report(y_test, y_pred))
print(sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='.4g',                 
                 xticklabels=['Normal behaviour (0 class)','Otherwise (1 class)'],                
                 yticklabels=['Normal behaviour (0 class)','Otherwise (1 class)'],                  
                 cbar=False, cmap='Blues'))

We infer that with this model over 1984 **0 class** values 46 were predicted as False Positive and over 224 **1 class** values 2 were predicted as False Negative.

In the table below some main comparative characteristics are presented:


| <center> **Comparative characteristics**  <center>   |  <center> **LogisticRegression** <center>  |  <center> **DecisionTreeClassifier**  <center> |  
| ----------- | ----------- | ----------- |
| <center> Number of used hyperparameters  <center>    | <center> 3  <center>     | <center> 2<center> | 
| <center> Running time <center>  | <center> ~ 5 sec <center>        | <center> ~ 5 sec <center> | 
| <center> Training accuracy  <center>  | <center> 0.979 <center>        | <center> 0.984 <center> | 
| <center> Testing accuracy  <center>  | <center> 0.975 <center>        | <center> 0.978 <center> | 
| <center> Type I error  <center>  | <center> 55 <center>        | <center> 46 <center> |
| <center> Type II error  <center>  | <center> 1 <center>        | <center> 2 <center> |
    
We can conclude, that overall both models perfomed very well, however the DecisionTreeClassifier perfomed slightly better.

### 5. Evaluation

The Evaluation phase is different from the Modeling technical evaluation. This phase evaluates the model concerning the business indicator and what to do next.
On the business understanding stage we've defined two main question we would like to get answers with the project:
* What features are the most important for bidding class prediction?
* Can a target variable be predicted with an accuracy higher than 90%?

We detected that three the most important features for predicting the target variable are:

* **Successive_Outbidding**;
* **Auction_Duration**;
* **Winning_Ratio**,

and the less important are:

* **Starting_Price_Average**;
* **Auction_ID**;
* **Early_Bidding**.

It means that next time when mining data we can focus more on getting observations for top 3 most important features and don't give great importance the less important features.

Also we found out that the target variable can be predicted with an accuracy higher than 90% if using the LogisticRegression and the DecisionTreeClassifier models.

### 6. Deployment

Although on practice the stage is being used as a way to deliver findings and outcomes to the user, I'll use the part to conclude what I've done.
So far, I've been able to deliver business goals, I had defined on the business understanding stage. I determined most important features which can be used in future for data mining. Both of model had perfomed really well and satisfied the accuracy requirements. I tuned hyperparameters and compared perfomance of two models. I was able to implement two dimensionality reduction techniques and compared the results.